In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import re
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [33]:
PATH = 'C:/Users/davin/Downloads/'

person = PATH + '202302_202302_주민등록인구및세대현황_월간.csv'
land   = PATH + '행정구역_20230318172834.csv'

In [102]:
person_df = pd.read_csv(person, encoding='cp949')
person_df[['구', '2023년02월_총인구수', '2023년02월_세대수', '2023년02월_세대당 인구', '2023년02월_여자 인구수', '2023년02월_남자 인구수']]

,구,2023년02월_총인구수,2023년02월_세대수,2023년02월_세대당 인구,2023년02월_여자 인구수,2023년02월_남자 인구수
0,NaN,"9,427,583","4,457,982",2.11,"4,859,626","4,567,957"
1,종로구,"141,433","72,773",1.94,"73,034","68,399"
2,중구,"120,555","63,492",1.90,"61,998","58,557"
3,용산구,"218,175","109,778",1.99,"113,315","104,860"
4,성동구,"280,474","133,517",2.10,"144,113","136,361"
5,광진구,"337,071","169,648",1.99,"174,742","162,329"
6,동대문구,"337,293","170,766",1.98,"171,331","165,962"
7,중랑구,"385,129","187,776",2.05,"195,823","189,306"
8,성북구,"430,170","197,673",2.18,"223,417","206,753"
9,강북구,"293,119","144,506",2.03,"150,865","142,254"


In [35]:
person_df.sort_values('2023년02월_총인구수', ascending=False)[:6][['구', '2023년02월_총인구수']]

,구,2023년02월_총인구수
0,NaN,"9,427,583"
24,송파구,"658,868"
16,강서구,"568,666"
23,강남구,"529,962"
11,노원구,"502,820"
21,관악구,"487,430"


In [36]:
land_df = pd.read_csv(land)
land_df[:6]

,자치구별(1),자치구별(2),면적 (km²),구성비 (%),행정 (개),법정 (개),소계,소계.1
0,서울시,소계,605.24,100.00,426,466,12859,96471
1,서울시,종로구,23.91,3.95,17,87,279,1475
2,서울시,중구,9.96,1.65,15,74,253,1580
3,서울시,용산구,21.87,3.61,16,36,356,2597
4,서울시,성동구,16.86,2.79,17,17,471,3645
5,서울시,광진구,17.06,2.82,15,7,370,3012


In [37]:
land_df.sort_values('면적 (km²)', ascending=False)[:6][['자치구별(2)', '면적 (km²)']]

,자치구별(2),면적 (km²)
0,소계,605.24
22,서초구,46.98
16,강서구,41.45
23,강남구,39.50
11,노원구,35.44
24,송파구,33.88


In [101]:
merge_outer = pd.merge(person_df, land_df, how='outer',left_on='구', right_on='자치구별(2)')
merge_outer[['구', '2023년02월_총인구수', '면적 (km²)']].head()

,구,2023년02월_총인구수,면적 (km²)
0,NaN,"9,427,583",NaN
1,종로구,"141,433",23.91
2,중구,"120,555",9.96
3,용산구,"218,175",21.87
4,성동구,"280,474",16.86


In [99]:
# merge_outer['per_per_land'] = merge_outer['2023년02월_총인구수']/merge_outer['면적 (km²)']
merge_outer['총인구수'] = merge_outer['2023년02월_총인구수']
for i in range(len(merge_outer['총인구수'])):
    try:
        merge_outer['총인구수'][i] = int(merge_outer['총인구수'][i].replace(',',''))
    except:
        continue

<ipython-input-99-db288658fcc6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_outer['총인구수'][i] = int(merge_outer['총인구수'][i].replace(',',''))


In [ ]:
merge_outer['인구밀집도'] = merge_outer['총인구수']/merge_outer['면적 (km²)']

In [ ]:
merge_outer['순위'] = merge_outer['인구밀집도']

In [98]:
# merge_outer.sort_values('인구밀집도', ascending=False)[['구', '2023년02월_총인구수', '면적 (km²)', '인구밀집도']][:6]
merge_outer.sort_values('면적 (km²)', ascending=False)[['구', '총인구수', '면적 (km²)', '인구밀집도']].sort_values('인구밀집도', ascending=False)

,구,총인구수,면적 (km²),인구밀집도
15,양천구,440042,17.41,25275.2
6,동대문구,337293,14.22,23719.6
20,동작구,379622,16.36,23204.3
7,중랑구,385129,18.50,20817.8
5,광진구,337071,17.06,19758
17,구로구,395422,20.12,19653.2
24,송파구,658868,33.88,19447.1
25,강동구,460280,24.59,18718.2
18,금천구,229395,13.02,17618.7
8,성북구,430170,24.58,17500.8
